# 低阈值与高阈值钙通道的门控特性对比

本教程将通过一个完整示例，展示如何使用 Braincell 库分析两种典型钙通道（低阈值 T 型钙通道和高阈值钙通道）的门控变量稳态特性。通过对比它们的门控曲线，你将理解不同钙通道在神经元电活动中的功能差异。


## 教程目标
- 学习如何在 Braincell 中调用预定义的离子通道模型。
- 理解钙通道门控变量的稳态曲线（`q_inf`、`p_inf`）的物理意义。
- 对比低阈值与高阈值钙通道的门控特性差异，及其与神经元功能的关联。


## 准备工作
首先确保已安装必要的库（`braincell`、`brainunit`、`braintools`、`matplotlib`），并导入所需模块：

```python
import braintools
import brainunit as u  # 神经科学常用单位
import matplotlib.pyplot as plt  # 基础绘图库
import braincell  # 核心库:用于构建和模拟离子通道
```


## 代码详解

### 创建钙通道模型
Braincell 内置了多种基于经典文献的离子通道模型，命名格式为 `通道类型_文献标识`（如 `ICaT_HP1992` 表示基于 Huguenard & Prince 1992 年研究的 T 型钙通道）。

```python
# 创建低阈值 T 型钙通道（ICaT），参数为电导密度（这里设为 1 mS/cm²）
# HP1992 表示模型来自 Huguenard & Prince (1992) 的研究
cat = braincell.channel.ICaT_HP1992(1)

# 创建高阈值钙通道（ICaHT），同样设置电导密度为 1 mS/cm²
# HM1992 表示模型来自 Huguenard & McCormick (1992) 的研究
caht = braincell.channel.ICaHT_HM1992(1)
```


### 生成电压序列
为了分析通道门控特性随膜电位的变化，我们生成一个从 -100 mV 到 0 mV 的连续电压序列：

```python
# 生成电压序列：从 -100 mV 到 0 mV，步长 0.1 mV
# brainunit 的 arange 函数支持直接带单位计算，避免单位换算错误
vs = u.math.arange(-100 * u.mV, 0 * u.mV, 0.1 * u.mV)
```


### 计算门控变量稳态值并绘图
离子通道的门控变量（如 `q`、`p`）的稳态值（`_inf`）描述了在某一膜电位下，通道门控处于“开放”状态的概率。我们通过通道的 `f_q_inf` 和 `f_p_inf` 方法计算这些值，并可视化对比：

```python
# 创建图形和子图布局（1 行 2 列，figsize 控制整体大小）
fig, gs = braintools.visualize.get_figure(1, 2, 3., 4.5)

# ---------------------- 绘制低阈值钙通道（ICaT）的门控曲线 ----------------------
# 计算低阈值通道的 q_inf（激活门稳态值）和 p_inf（失活门稳态值）
q_inf = cat.f_q_inf(vs)
p_inf = cat.f_p_inf(vs)

# 添加子图 1（第 1 行第 1 列）
fig.add_subplot(gs[0, 0])
plt.plot(vs / u.mV, q_inf, label='q_inf（激活门）')  # 横轴转换为 mV 便于阅读
plt.plot(vs / u.mV, p_inf, label='p_inf（失活门）')
plt.legend()  # 显示图例
plt.fill_between([-80, -60], 1., alpha=0.2)  # 标记低阈值通道的典型激活范围
plt.title('低阈值钙通道（ICaT）')
plt.xlabel('膜电位（mV）')

# ---------------------- 绘制高阈值钙通道（ICaHT）的门控曲线 ----------------------
# 计算高阈值通道的 q_inf 和 p_inf
q_inf = caht.f_q_inf(vs)
p_inf = caht.f_p_inf(vs)

# 添加子图 2（第 1 行第 2 列）
fig.add_subplot(gs[0, 1])
plt.plot(vs / u.mV, q_inf, label='q_inf（激活门）')
plt.plot(vs / u.mV, p_inf, label='p_inf（失活门）')
plt.fill_between([-60, -40], 1., alpha=0.2)  # 标记高阈值通道的典型激活范围
plt.legend()
plt.xlabel('膜电位（mV）')
plt.title('高阈值钙通道（ICaHT）')

# 显示图形
plt.show()
```


## 结果解读
运行代码后，你将看到两张门控特性曲线图，如下图：

分析两张门控特性曲线图，核心结论如下：
**低阈值钙通道（左图）**：
   - `q_inf`（激活门）在膜电位约 -80 ~ -60 mV 时显著上升，表明此范围内通道易激活。
   - 与填充区域（-80 ~ -60 mV）吻合，说明该通道在神经元静息电位附近（较负电位）即可被激活。
**高阈值钙通道（右图）**：
   - `q_inf` 在膜电位约 -60 ~ -40 mV 时才显著上升，激活阈值更高。
   - 填充区域（-60 ~ -40 mV）对应其典型激活范围，需更强去极化才能激活。



## 总结与扩展
本教程通过 Braincell 的预定义通道模型，快速对比了两种钙通道的门控特性，展示了：
- 如何调用文献中的经典通道模型（`ICaT_HP1992`、`ICaHT_HM1992`）；
- 门控变量稳态曲线（`_inf`）的计算与可视化方法；
- 低/高阈值钙通道的功能差异（激活阈值不同）。

**扩展练习**：
- 尝试修改通道的电导密度参数（如 `ICaT_HP1992(0.5)`），观察门控曲线是否变化（提示：稳态曲线与电导无关，仅与电压相关）。
- 对比其他离子通道（如钠通道、钾通道）的门控特性，理解不同通道在动作电位中的协同作用。

通过这些工具，你可以快速构建符合生物物理特性的神经元模型，为复杂神经动力学模拟奠定基础。